In [1]:
!pip install pykalman

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 3.7 MB/s eta 0:00:00


## 더미변수 포함

## 패널분석 (랜덤 -> fixed)

In [73]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter


In [81]:
data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')

<ipython-input-81-58ee1529768c>:1: DtypeWarning: Columns (15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')


In [82]:
# Convert Date to datetime format for proper sorting
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Sort the data by Date, AppName, and Country
data = data.sort_values(by=['AppName', 'Country', 'Date'])

# Group by AppName and Country, then calculate the cumulative mean for Daily Average Rating
data['Cumulative_Avg_Rating'] = data.groupby(['AppName', 'Country'])['Daily Average Rating'].expanding().mean().reset_index(level=[0,1], drop=True)

# Replace missing values in Total Average Rating with the cumulative average
data['Total Average Rating'].fillna(data['Cumulative_Avg_Rating'], inplace=True)

# Drop the helper column 'Cumulative_Avg_Rating' after imputation
data.drop(columns=['Cumulative_Avg_Rating'], inplace=True)



<ipython-input-82-6b0d2e921bad>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Cumulative Daily Average Rating'].fillna(data['Cumulative Daily Average Rating'].mean(), inplace=True)
<ipython-input-82-6b0d2e921bad>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

In [83]:
data.drop(columns = 'Daily Average Rating',axis = 1, inplace = True)

In [152]:
data['Language_Game_Flag'].value_counts()

,count
Language_Game_Flag,
0.000000,288561
0.693147,3248


In [84]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 291809 entries, 45209 to 291628
Data columns (total 40 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Date                                   291809 non-null  object 
 1   AppName                                291809 non-null  object 
 2   Country                                291809 non-null  object 
 3   CONTINENT                              291809 non-null  object 
 4   AD_LAN                                 291809 non-null  object 
 5   Store_Visit                            291809 non-null  int64  
 6   Store_Acquisition                      291809 non-null  int64  
 7   DAU                                    291809 non-null  float64
 8   MAU                                    291809 non-null  float64
 9   Bound                                  291809 non-null  float64
 10  User acquisition                       291809 non-null  f

In [85]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User acquisition'])

# 새로운 YouTube_Flag 생성: _youtube가 포함되지만 unsimilar_youtube 제외
youtube_columns = data.filter(like='_youtube').columns
youtube_columns = youtube_columns[~youtube_columns.str.contains('unsimilar')]
data['YouTube_Flag'] = data[youtube_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# 새로운 Games_Flag 생성: _games가 포함되지만 previous_games 제외
games_columns = data.filter(like='_games').columns
games_columns = games_columns[~games_columns.str.contains('previous')]
data['Games_Flag'] = data[games_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AppName_original'] = data['AppName']
data['Country_original'] = data['Country']
data['AD_LAN_original'] = data['AD_LAN']

# 변수 이름의 공백을 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_')

# 범주형 변수를 더미 변수로 변환 (Country와 다른 범주형 변수들)
data = pd.get_dummies(data, columns=['Country', 'AppName', 'CONTINENT', 'AD_LAN'], drop_first=True)

# Language_Game_Flag 적용
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN_original'] == '한국어' or row['AD_LAN_original'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['Store_Visit', 'Store_Acquisition',
       'View_Count_games', 'similarity_score', 'View_Count_youtube',
       'Like_Count_youtube', 'Comment_Count_youtube', 'video_potentiality_youtube',
       'video_potentiality_games', 'day+_youtube', 'day+_games', 'Category_ID_youtube',
       'Category_ID_games', 'similarity_score_games', 'Total_Average_Rating',
       'View_Count_previous_games', 'video_potentiality_previous_games',
       'day+_unsimilar_youtube', 'video_potentiality_unsimilar_youtube',
       'Comment_Count_Count_unsimilar_youtube', 'View_Count_unsimilar_youtube',
       'Like_Count_unsimilar_youtube', 'YouTube_Flag', 'Games_Flag', 'Language_Game_Flag']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# 스케일링 적용
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF 결과 출력
print(vif_data)

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
print("VIF가 낮은 독립변수들:", low_vif_features)

# VIF가 높은 변수를 제거한 후 Kalman 필터 및 회귀 모델 분석
# 칼만 필터 및 회귀 분석 코드 그대로 유지
user_acquisition = data['User_acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# Log 변환 (0보다 큰 값에만 적용)
for col in ['Permanent_Component', 'Temporary_Component'] + low_vif_features:
    if (data[col] > 0).all():
        data[col] = np.log(data[col])

# 패널 데이터를 위한 필요한 독립 변수만 선택 (low_vif_features 및 원본 AppName, Country, AD_LAN 사용)
panel_data = data[['Date', 'AppName_original', 'Country_original', 'Permanent_Component', 'Temporary_Component'] + low_vif_features + list(data.filter(like='AD_LAN').columns)]

# 결측값 제거
panel_data = panel_data.dropna()

# 변수명에서 +를 _로 교체
panel_data.columns = panel_data.columns.str.replace('+', '_')
low_vif_features = [feature.replace('+', '_') for feature in low_vif_features]

# 고정효과 패널 회귀 분석 (Permanent Component 분석 - log-log)
model_perm = smf.ols(
    'Permanent_Component ~ ' + ' + '.join(low_vif_features) +
    ' + C(AppName_original) + C(Country_original) + ' +
    ' + '.join(list(panel_data.filter(like="AD_LAN").columns)),
    data=panel_data
).fit()

# 결과 출력
print("고정효과 패널 분석 결과 (Permanent Component - log-log):")
print(model_perm.summary())

# 고정효과 패널 회귀 분석 (Temporary Component 분석 - log-log)
model_temp = smf.ols(
    'Temporary_Component ~ ' + ' + '.join(low_vif_features) +
    ' + C(AppName_original) + C(Country_original) + ' +
    ' + '.join(list(panel_data.filter(like="AD_LAN").columns)),
    data=panel_data
).fit()

# 결과 출력
print("\n고정효과 패널 분석 결과 (Temporary Component - log-log):")
print(model_temp.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


                                  Feature         VIF
0                                   const         NaN
1                             Store_Visit   19.420397
2                       Store_Acquisition   19.387839
3                        View_Count_games   10.947437
4                        similarity_score    6.996719
5                      View_Count_youtube    1.621433
6                      Like_Count_youtube    1.085381
7                   Comment_Count_youtube         NaN
8              video_potentiality_youtube    1.580665
9                video_potentiality_games   11.697279
10                           day+_youtube    4.259015
11                             day+_games   10.735712
12                    Category_ID_youtube    2.627060
13                      Category_ID_games  165.699193
14                 similarity_score_games   10.930804
15                   Total_Average_Rating    1.006566
16              View_Count_previous_games    6.974691
17      video_potentiality_p

### 더미변수 제외

In [107]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 291809 entries, 0 to 291808
Data columns (total 39 columns):
 #   Column                                 Non-Null Count   Dtype         
---  ------                                 --------------   -----         
 0   Date                                   291809 non-null  datetime64[ns]
 1   AppName                                291809 non-null  object        
 2   Country                                291809 non-null  object        
 3   CONTINENT                              291809 non-null  object        
 4   AD_LAN                                 291809 non-null  object        
 5   Store_Visit                            291809 non-null  int64         
 6   Store_Acquisition                      291809 non-null  int64         
 7   DAU                                    291809 non-null  float64       
 8   MAU                                    291809 non-null  float64       
 9   Bound                                  291809 non-nul

In [149]:
import pandas as pd

# Load the data with low_memory=False to prevent DtypeWarning
data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv', low_memory=False)

# Convert Date to datetime format for proper sorting
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Sort the data by Date and Country
data = data.sort_values(by=['Country', 'Date'])

# Step 1: Replace 0s and NaNs in 'Daily Average Rating' with the most recent non-zero value within each 'Country' group
def replace_zero_and_nan(series):
    series = series.replace(0, pd.NA)  # Replace 0 with NaN
    return series.ffill()  # Forward fill

# Apply replace_zero_and_nan within each group
data['Daily Average Rating'] = data.groupby('Country')['Daily Average Rating'].transform(replace_zero_and_nan)

# Step 2: Calculate the cumulative mean for 'Daily Average Rating' by 'Country'
# We use `transform` here to avoid issues with incompatible indices
data['Cumulative_Avg_Rating'] = data.groupby('Country')['Daily Average Rating'].transform(lambda x: x.expanding().mean())

# Step 3: Replace missing values in 'Total Average Rating' with the cumulative average
data['Total Average Rating'] = data['Total Average Rating'].fillna(data['Cumulative_Avg_Rating'])

# Step 4: If there are still missing values in 'Total Average Rating', fill them with the country-wise mean
data['Total Average Rating'] = data.groupby('Country')['Total Average Rating'].transform(lambda x: x.fillna(x.mean()))

# Optional Step: If some values are still NaN, fill them with the global mean or a constant value
data['Total Average Rating'].fillna(data['Total Average Rating'].mean(), inplace=True)

# Clean up: Drop the temporary 'Cumulative_Avg_Rating' column
data.drop(columns=['Cumulative_Avg_Rating'], inplace=True)

# Save the processed data (optional)
# data.to_csv('/path/to/save/imputed_data.csv', index=False)

<ipython-input-149-f4851dfcda7a>:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return series.ffill()  # Forward fill
<ipython-input-149-f4851dfcda7a>:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Total Average Rating'].fillna(data['Total Average Rating'].mean(), inplace=True)


In [151]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from pykalman import KalmanFilter

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User acquisition'])

# 새로운 YouTube_Flag 생성: _youtube가 포함되지만 unsimilar_youtube 제외
youtube_columns = data.filter(like='_youtube').columns
youtube_columns = youtube_columns[~youtube_columns.str.contains('unsimilar')]
data['YouTube_Flag'] = data[youtube_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# 새로운 Games_Flag 생성: _games가 포함되지만 previous_games 제외
games_columns = data.filter(like='_games').columns
games_columns = games_columns[~games_columns.str.contains('previous')]
data['Games_Flag'] = data[games_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AppName_original'] = data['AppName']
data['Country_original'] = data['Country']
data['AD_LAN_original'] = data['AD_LAN']

# 변수 이름의 공백을 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_')

# 범주형 변수를 더미 변수로 변환 (Country와 다른 범주형 변수들)
data = pd.get_dummies(data, columns=['Country', 'AppName', 'CONTINENT', 'AD_LAN'], drop_first=True)

# Language_Game_Flag 적용
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN_original'] == '한국어' or row['AD_LAN_original'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['Store_Visit',
       'View_Count_games',
       'similarity_score', 'View_Count_youtube', 'Like_Count_youtube',
       'Comment_Count_youtube', 'video_potentiality_youtube',
       'video_potentiality_games', 'day+_youtube', 'day+_games',
       'Category_ID_youtube', 'Category_ID_games', 'similarity_score_games',
       'Total_Average_Rating', 'View_Count_previous_games', 'video_potentiality_previous_games',
       'day+_unsimilar_youtube', 'video_potentiality_unsimilar_youtube',
       'Comment_Count_Count_unsimilar_youtube', 'View_Count_unsimilar_youtube',
       'Like_Count_unsimilar_youtube', 'YouTube_Flag', 'Games_Flag', 'Language_Game_Flag']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# VIF 계산 없이 low_vif_features는 그대로 사용
low_vif_features = independent_vars

# Kalman Filter 설정
user_acquisition = data['User_acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# Log 변환 (값에 1을 더한 후 0보다 큰 값에만 적용)
for col in ['Permanent_Component', 'Temporary_Component'] + low_vif_features:
    data[col] = np.log(data[col] + 1)

# 패널 데이터를 위한 필요한 독립 변수만 선택 (low_vif_features 및 원본 AppName, Country, AD_LAN 사용)
panel_data = data[low_vif_features + ['Permanent_Component', 'Temporary_Component']]

# 결측값 제거
panel_data = panel_data.dropna()

# 변수명에서 +를 _로 교체
panel_data.columns = panel_data.columns.str.replace('+', '_')
low_vif_features = [feature.replace('+', '_') for feature in low_vif_features]

# 고정효과 패널 회귀 분석 (Permanent Component 분석 - log-log)
model_perm = smf.ols('Permanent_Component ~ ' + ' + '.join(low_vif_features), data=panel_data).fit()

# 결과 출력
print("고정효과 패널 분석 결과 (Permanent Component - log-log):")
print(model_perm.summary())

# 고정효과 패널 회귀 분석 (Temporary Component 분석 - log-log)
model_temp = smf.ols('Temporary_Component ~ ' + ' + '.join(low_vif_features), data=panel_data).fit()

# 결과 출력
print("\n고정효과 패널 분석 결과 (Temporary Component - log-log):")
print(model_temp.summary())

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


고정효과 패널 분석 결과 (Permanent Component - log-log):
                             OLS Regression Results                            
Dep. Variable:     Permanent_Component   R-squared:                       0.914
Model:                             OLS   Adj. R-squared:                  0.914
Method:                  Least Squares   F-statistic:                 6.607e+04
Date:                 Wed, 23 Oct 2024   Prob (F-statistic):               0.00
Time:                         12:25:51   Log-Likelihood:            -1.1634e+05
No. Observations:               136229   AIC:                         2.327e+05
Df Residuals:                   136206   BIC:                         2.330e+05
Df Model:                           22                                         
Covariance Type:             nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------

### VIF 무시
#### independent_vars의 내용물을 수정해서 관찰해보세요~~~

In [140]:
import pandas as pd

# Load the data with low_memory=False to prevent DtypeWarning
data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv', low_memory=False)

# Convert Date to datetime format for proper sorting
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Sort the data by Date and Country
data = data.sort_values(by=['Country', 'Date'])

# Step 1: Replace 0s and NaNs in 'Daily Average Rating' with the most recent non-zero value within each 'Country' group
def replace_zero_and_nan(series):
    series = series.replace(0, pd.NA)  # Replace 0 with NaN
    return series.ffill()  # Forward fill

# Apply replace_zero_and_nan within each group
data['Daily Average Rating'] = data.groupby('Country')['Daily Average Rating'].transform(replace_zero_and_nan)

# Step 2: Calculate the cumulative mean for 'Daily Average Rating' by 'Country'
# We use `transform` here to avoid issues with incompatible indices
data['Cumulative_Avg_Rating'] = data.groupby('Country')['Daily Average Rating'].transform(lambda x: x.expanding().mean())

# Step 3: Replace missing values in 'Total Average Rating' with the cumulative average
data['Total Average Rating'] = data['Total Average Rating'].fillna(data['Cumulative_Avg_Rating'])

# Step 4: If there are still missing values in 'Total Average Rating', fill them with the country-wise mean
data['Total Average Rating'] = data.groupby('Country')['Total Average Rating'].transform(lambda x: x.fillna(x.mean()))

# Optional Step: If some values are still NaN, fill them with the global mean or a constant value
data['Total Average Rating'].fillna(data['Total Average Rating'].mean(), inplace=True)

# Clean up: Drop the temporary 'Cumulative_Avg_Rating' column
data.drop(columns=['Cumulative_Avg_Rating'], inplace=True)

# Save the processed data (optional)
# data.to_csv('/path/to/save/imputed_data.csv', index=False)

<ipython-input-140-f4851dfcda7a>:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return series.ffill()  # Forward fill
<ipython-input-140-f4851dfcda7a>:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Total Average Rating'].fillna(data['Total Average Rating'].mean(), inplace=True)


In [142]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User acquisition'])

# 새로운 YouTube_Flag 생성: _youtube가 포함되지만 unsimilar_youtube 제외
youtube_columns = data.filter(like='_youtube').columns
youtube_columns = youtube_columns[~youtube_columns.str.contains('unsimilar')]
data['YouTube_Flag'] = data[youtube_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# 새로운 Games_Flag 생성: _games가 포함되지만 previous_games 제외
games_columns = data.filter(like='_games').columns
games_columns = games_columns[~games_columns.str.contains('previous')]
data['Games_Flag'] = data[games_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AppName_original'] = data['AppName']
data['Country_original'] = data['Country']
data['AD_LAN_original'] = data['AD_LAN']

# 변수 이름의 공백과 특수문자 '+'를 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_').str.replace('+', '_')

# 범주형 변수를 더미 변수로 변환 (Country와 다른 범주형 변수들)
data = pd.get_dummies(data, columns=['Country', 'AppName', 'CONTINENT', 'AD_LAN'], drop_first=True)

# Language_Game_Flag 적용
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN_original'] == '한국어' or row['AD_LAN_original'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# VIF가 낮은 독립변수들만 사용
independent_vars = ['similarity_score', 'View_Count_youtube', 'Like_Count_youtube',
                    'video_potentiality_youtube', 'day__youtube', 'Category_ID_youtube',
                    'Total_Average_Rating', 'View_Count_previous_games', 'video_potentiality_previous_games',
                    'day__unsimilar_youtube', 'video_potentiality_unsimilar_youtube',
                    'View_Count_unsimilar_youtube', 'Like_Count_unsimilar_youtube',
                    'YouTube_Flag', 'Language_Game_Flag']

# Kalman Filter 설정
user_acquisition = data['User_acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 로그 변환을 적용해야 하는 변수들을 선택 (수치형 변수에만 적용, 더미 변수 제외)
log_transformed_vars = [var for var in independent_vars if var not in ['YouTube_Flag', 'Language_Game_Flag']]

# 로그 변환: 0이나 음수는 변환하지 않도록 처리 (log(0) undefined). 매우 작은 값을 추가하여 처리
for var in log_transformed_vars:
    data[var] = data[var].apply(lambda x: np.log(x + 1e-6) if x > 0 else 0)  # 0에 1e-6을 더해 로그를 취함

# Permanent 및 Temporary Components에도 로그 변환 적용
data['log_Permanent_Component'] = np.log(data['Permanent_Component'].replace(0, np.nan) + 1e-6)
data['log_Temporary_Component'] = np.log(data['Temporary_Component'].replace(0, np.nan) + 1e-6)

# 고정효과 패널 회귀 분석 (로그 Permanent Component 분석)
model_perm = smf.ols('log_Permanent_Component ~ ' + ' + '.join(log_transformed_vars + ['YouTube_Flag', 'Language_Game_Flag']), data=data).fit()

# 결과 출력
print("고정효과 패널 분석 결과 (Log Permanent Component):")
print(model_perm.summary())

# 고정효과 패널 회귀 분석 (로그 Temporary Component 분석)
model_temp = smf.ols('log_Temporary_Component ~ ' + ' + '.join(log_transformed_vars + ['YouTube_Flag', 'Language_Game_Flag']), data=data).fit()

# 결과 출력
print("\n고정효과 패널 분석 결과 (Log Temporary Component):")
print(model_temp.summary())

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


고정효과 패널 분석 결과 (Log Permanent Component):
                               OLS Regression Results                              
Dep. Variable:     log_Permanent_Component   R-squared:                       0.014
Model:                                 OLS   Adj. R-squared:                  0.014
Method:                      Least Squares   F-statistic:                     274.8
Date:                     Wed, 23 Oct 2024   Prob (F-statistic):               0.00
Time:                             12:04:05   Log-Likelihood:            -6.2555e+05
No. Observations:                   291809   AIC:                         1.251e+06
Df Residuals:                       291793   BIC:                         1.251e+06
Df Model:                               15                                         
Covariance Type:                 nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------

### Segment 요소 추가

In [133]:
import pandas as pd

# Load the data with low_memory=False to prevent DtypeWarning
data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv', low_memory=False)

# Convert Date to datetime format for proper sorting
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Sort the data by Date and Country
data = data.sort_values(by=['Country', 'Date'])

# Step 1: Replace 0s and NaNs in 'Daily Average Rating' with the most recent non-zero value within each 'Country' group
def replace_zero_and_nan(series):
    series = series.replace(0, pd.NA)  # Replace 0 with NaN
    return series.ffill()  # Forward fill

# Apply replace_zero_and_nan within each group
data['Daily Average Rating'] = data.groupby('Country')['Daily Average Rating'].transform(replace_zero_and_nan)

# Step 2: Calculate the cumulative mean for 'Daily Average Rating' by 'Country'
# We use `transform` here to avoid issues with incompatible indices
data['Cumulative_Avg_Rating'] = data.groupby('Country')['Daily Average Rating'].transform(lambda x: x.expanding().mean())

# Step 3: Replace missing values in 'Total Average Rating' with the cumulative average
data['Total Average Rating'] = data['Total Average Rating'].fillna(data['Cumulative_Avg_Rating'])

# Step 4: If there are still missing values in 'Total Average Rating', fill them with the country-wise mean
data['Total Average Rating'] = data.groupby('Country')['Total Average Rating'].transform(lambda x: x.fillna(x.mean()))

# Optional Step: If some values are still NaN, fill them with the global mean or a constant value
data['Total Average Rating'].fillna(data['Total Average Rating'].mean(), inplace=True)

# Clean up: Drop the temporary 'Cumulative_Avg_Rating' column
data.drop(columns=['Cumulative_Avg_Rating'], inplace=True)

# Save the processed data (optional)
# data.to_csv('/path/to/save/imputed_data.csv', index=False)

<ipython-input-133-f4851dfcda7a>:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return series.ffill()  # Forward fill
<ipython-input-133-f4851dfcda7a>:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Total Average Rating'].fillna(data['Total Average Rating'].mean(), inplace=True)


In [134]:
segment = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/segment.csv')

In [135]:
segment_cocobi = pd.merge(data, segment, on = 'Country', how = 'left')

In [136]:
data = segment_cocobi.copy()

segment와

In [137]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291809 entries, 0 to 291808
Data columns (total 40 columns):
 #   Column                                 Non-Null Count   Dtype         
---  ------                                 --------------   -----         
 0   Date                                   291809 non-null  datetime64[ns]
 1   AppName                                291809 non-null  object        
 2   Country                                291809 non-null  object        
 3   CONTINENT                              291809 non-null  object        
 4   AD_LAN                                 291809 non-null  object        
 5   Store_Visit                            291809 non-null  int64         
 6   Store_Acquisition                      291809 non-null  int64         
 7   DAU                                    291809 non-null  float64       
 8   MAU                                    291809 non-null  float64       
 9   Bound                                  291809 no

### segment 1을 기준으로 segment를 더미변수로 지정한 다음 다중회귀분석

In [139]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User acquisition'])

# 새로운 YouTube_Flag 생성: _youtube가 포함되지만 unsimilar_youtube 제외
youtube_columns = data.filter(like='_youtube').columns
youtube_columns = youtube_columns[~youtube_columns.str.contains('unsimilar')]
data['YouTube_Flag'] = data[youtube_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# 새로운 Games_Flag 생성: _games가 포함되지만 previous_games 제외
games_columns = data.filter(like='_games').columns
games_columns = games_columns[~games_columns.str.contains('previous')]
data['Games_Flag'] = data[games_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AD_LAN_original'] = data['AD_LAN']
data['segment_original'] = data['segment']

# 변수 이름의 공백과 특수문자 '+'를 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_').str.replace('+', '_')

# Check if 'segment' column exists before converting to dummy variables
if 'segment' in data.columns:
    data = pd.get_dummies(data, columns=['segment'], drop_first=False)
else:
    print("'segment' column not found, skipping dummy conversion for 'segment'.")

# Language_Game_Flag 적용 함수
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN_original'] == '한국어' or row['AD_LAN_original'] == '영어')
    if is_korean_or_english and row['video_potentiality_games'] > 0:
        return 1
    else:
        return 0

# Language_Game_Flag 적용
data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['Store_Visit', 'View_Count_games', 'similarity_score', 'View_Count_youtube',
                    'Like_Count_youtube', 'Comment_Count_youtube', 'video_potentiality_youtube',
                    'video_potentiality_games', 'day__youtube', 'day__games', 'Category_ID_youtube',
                    'Category_ID_games', 'similarity_score_games', 'Total_Average_Rating',
                    'View_Count_previous_games', 'video_potentiality_previous_games',
                    'day__unsimilar_youtube', 'video_potentiality_unsimilar_youtube',
                    'Comment_Count_Count_unsimilar_youtube', 'View_Count_unsimilar_youtube',
                    'Like_Count_unsimilar_youtube', 'YouTube_Flag', 'Games_Flag', 'Language_Game_Flag']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars + list(data.filter(like='segment').columns)] if 'segment' in data.columns else data[independent_vars]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# 스케일링 적용
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
print("VIF가 낮은 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data['User_acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 종속 변수에만 로그 변환 적용 (독립 변수는 더미 변수를 제외하고 변환)
data['log_Permanent_Component'] = np.log(data['Permanent_Component'].replace(0, np.nan))
data['log_Temporary_Component'] = np.log(data['Temporary_Component'].replace(0, np.nan))

# 더미 변수는 로그 변환을 하지 않음 (YouTube_Flag, Games_Flag, Language_Game_Flag는 그대로 유지)
log_transformed_vars = [var for var in low_vif_features if var not in ['YouTube_Flag', 'Games_Flag', 'Language_Game_Flag']]

# 연속형 독립 변수에 로그 변환 적용
for var in log_transformed_vars:
    data[var] = data[var].apply(lambda x: np.log(x) if x > 0 else 0)

# 상수항 추가 (Intercept)
data = add_constant(data)

# 패널 데이터를 위한 필요한 독립 변수만 선택 (low_vif_features 및 원본 AppName, Country, AD_LAN 사용)
panel_data = data[['Date', 'segment_original', 'log_Permanent_Component', 'log_Temporary_Component'] + low_vif_features]

# 결측값 제거
panel_data = panel_data.dropna()

# 변수명에서 +를 _로 교체
panel_data.columns = panel_data.columns.str.replace('+', '_')
low_vif_features = [feature.replace('+', '_') for feature in low_vif_features]

# 고정효과 패널 회귀 분석 (Log Permanent Component 분석) - segment와 Language_Game_Flag 상호작용 추가
model_perm = smf.ols('log_Permanent_Component ~ ' + ' + '.join(log_transformed_vars) +
                     ' + C(segment_original) * Language_Game_Flag', data=panel_data).fit()

# 고정효과 패널 회귀 분석 (Log Temporary Component 분석) - segment와 Language_Game_Flag 상호작용 추가
model_temp = smf.ols('log_Temporary_Component ~ ' + ' + '.join(log_transformed_vars) +
                     ' + C(segment_original) * Language_Game_Flag', data=panel_data).fit()

# 결과 출력
print("고정효과 패널 분석 결과 (Log Permanent Component):")
print(model_perm.summary())

print("\n고정효과 패널 분석 결과 (Log Temporary Component):")
print(model_temp.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


VIF 결과:
                                   Feature         VIF
0                                   const         NaN
1                             Store_Visit    1.011204
2                        View_Count_games   12.527266
3                        similarity_score    6.891744
4                      View_Count_youtube    1.817352
5                      Like_Count_youtube    1.080320
6                   Comment_Count_youtube         NaN
7              video_potentiality_youtube    1.773816
8                video_potentiality_games   13.497812
9                            day__youtube    4.235366
10                             day__games   11.187104
11                    Category_ID_youtube    2.748774
12                      Category_ID_games   92.749031
13                 similarity_score_games   11.752782
14                   Total_Average_Rating    1.004885
15              View_Count_previous_games    7.237799
16      video_potentiality_previous_games    7.239669
17                 

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


고정효과 패널 분석 결과 (Log Permanent Component):
                               OLS Regression Results                              
Dep. Variable:     log_Permanent_Component   R-squared:                       0.890
Model:                                 OLS   Adj. R-squared:                  0.890
Method:                      Least Squares   F-statistic:                 4.871e+04
Date:                     Wed, 23 Oct 2024   Prob (F-statistic):               0.00
Time:                             11:48:24   Log-Likelihood:                -97041.
No. Observations:                   108712   AIC:                         1.941e+05
Df Residuals:                       108693   BIC:                         1.943e+05
Df Model:                               18                                         
Covariance Type:                 nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


In [132]:
data

,const,Date,AppName,Country,CONTINENT,AD_LAN,Store_Visit,Store_Acquisition,DAU,MAU,...,segment_original,segment_1,segment_2,segment_3,segment_4,Language_Game_Flag,Permanent_Component,Temporary_Component,log_Permanent_Component,log_Temporary_Component
0,1.0,2021-07-29,퍼즐놀이,Argentina,중남미,스페인어,3.688879,1,0.0,0.0,...,3,False,False,True,False,0,1.000000,0.000000,0.000000,NaN
1,1.0,2021-07-30,퍼즐놀이,Argentina,중남미,스페인어,3.737670,6,0.0,0.0,...,3,False,False,True,False,0,2.490909,1.418182,0.912648,0.349376
2,1.0,2021-07-31,여름휴가,Argentina,중남미,스페인어,0.000000,0,0.0,0.0,...,3,False,False,True,False,0,1.155854,-0.107073,0.144839,NaN
3,1.0,2021-07-31,퍼즐놀이,Argentina,중남미,스페인어,3.526361,6,0.0,0.0,...,3,False,False,True,False,0,1.884072,0.593052,0.633435,-0.522473
4,1.0,2021-08-01,여름휴가,Argentina,중남미,스페인어,0.000000,0,0.0,0.0,...,3,False,False,True,False,0,1.004816,-0.341069,0.004805,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291804,1.0,2024-08-07,여름휴가,Uruguay,중남미,스페인어,2.564949,8,0.0,0.0,...,3,False,False,True,False,0,13.591476,-4.027630,2.609443,NaN
291805,1.0,2024-08-08,놀이공원,Uruguay,중남미,스페인어,2.639057,10,0.0,0.0,...,3,False,False,True,False,0,13.353994,-3.470861,2.591815,NaN
291806,1.0,2024-08-08,여름휴가,Uruguay,중남미,스페인어,2.639057,9,0.0,0.0,...,3,False,False,True,False,0,13.809094,-2.376307,2.625327,NaN
291807,1.0,2024-08-09,놀이공원,Uruguay,중남미,스페인어,2.302585,6,0.0,0.0,...,3,False,False,True,False,0,12.270609,-3.350797,2.507207,NaN


### 아래부터는 2,3,4를 각각 참조변수로 하여 진행한 분석

In [ ]:
# segment와 Language_Game_Flag기준 데이터 수(여기 나타나지 않으면 아예 데이터가 없는것)
data.groupby(['segment_original','Language_Game_Flag'])['const'].count()

segment_original  Language_Game_Flag
1                 0                       9441
                  1                        286
2                 0                      19454
3                 0                     106997
4                 0                     152669
                  1                       2962
Name: const, dtype: int64

In [ ]:
data = segment_cocobi.copy()

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter
from patsy.contrasts import Treatment

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User acquisition'])

# 새로운 YouTube_Flag 생성: _youtube가 포함되지만 unsimilar_youtube 제외
youtube_columns = data.filter(like='_youtube').columns
youtube_columns = youtube_columns[~youtube_columns.str.contains('unsimilar')]
data['YouTube_Flag'] = data[youtube_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# 새로운 Games_Flag 생성: _games가 포함되지만 previous_games 제외
games_columns = data.filter(like='_games').columns
games_columns = games_columns[~games_columns.str.contains('previous')]
data['Games_Flag'] = data[games_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AD_LAN_original'] = data['AD_LAN']
data['segment_original'] = data['segment']

# 변수 이름의 공백을 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_')

# Check if 'segment' column exists before converting to dummy variables
if 'segment' in data.columns:
    data = pd.get_dummies(data, columns=['segment'], drop_first=False)
else:
    print("'segment' column not found, skipping dummy conversion for 'segment'.")

# Language_Game_Flag 적용 함수
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN_original'] == '한국어' or row['AD_LAN_original'] == '영어')
    if is_korean_or_english and row['video_potentiality_games'] > 0:
        return 1
    else:
        return 0

# Language_Game_Flag 적용
data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['Store_Visit','View_Count_games',
       'similarity_score', 'View_Count_youtube', 'Like_Count_youtube',
       'Comment_Count_youtube', 'video_potentiality_youtube',
       'video_potentiality_games', 'day+_youtube', 'day+_games',
       'Category_ID_youtube', 'Category_ID_games', 'similarity_score_games',
      'Total_Average_Rating','View_Count_previous_games', 'video_potentiality_previous_games',
       'day+_unsimilar_youtube', 'video_potentiality_unsimilar_youtube',
       'Comment_Count_Count_unsimilar_youtube', 'View_Count_unsimilar_youtube',
       'Like_Count_unsimilar_youtube','YouTube_Flag','Games_Flag','Language_Game_Flag']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars +
         list(data.filter(like='segment').columns)] if 'segment' in data.columns else data[independent_vars]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# 스케일링 적용
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
print("VIF가 낮은 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data['User_acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# 패널 데이터를 위한 필요한 독립 변수만 선택 (low_vif_features 및 원본 AppName, Country, AD_LAN 사용)
panel_data = data[['Date','segment_original', 'Permanent_Component', 'Temporary_Component'] + low_vif_features]

# 결측값 제거
panel_data = panel_data.dropna()

# 변수명에서 +를 _로 교체
panel_data.columns = panel_data.columns.str.replace('+', '_')
low_vif_features = [feature.replace('+', '_') for feature in low_vif_features]


# 고정효과 패널 회귀 분석 (Permanent Component 분석) - segment와 Language_Game_Flag 상호작용 추가
model_perm = smf.ols('Permanent_Component ~ ' + ' + '.join(low_vif_features) +
                     ' + C(segment_original, Treatment(reference=2)) * Language_Game_Flag', data=panel_data).fit()

# 고정효과 패널 회귀 분석 (Temporary Component 분석) - segment와 Language_Game_Flag 상호작용 추가
model_temp = smf.ols('Temporary_Component ~ ' + ' + '.join(low_vif_features) +
                     ' + C(segment_original, Treatment(reference=2)) * Language_Game_Flag', data=panel_data).fit()

# 결과 출력
print("고정효과 패널 분석 결과 (Permanent Component):")
print(model_perm.summary())

print("\n고정효과 패널 분석 결과 (Temporary Component):")
print(model_temp.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


VIF 결과:
                                   Feature         VIF
0                                   const         NaN
1                             Store_Visit    1.013292
2                        View_Count_games   12.527275
3                        similarity_score    6.891993
4                      View_Count_youtube    1.817350
5                      Like_Count_youtube    1.080320
6                   Comment_Count_youtube         NaN
7              video_potentiality_youtube    1.773818
8                video_potentiality_games   13.497815
9                            day+_youtube    4.235413
10                             day+_games   11.187053
11                    Category_ID_youtube    2.748766
12                      Category_ID_games   92.749093
13                 similarity_score_games   11.752945
14                   Total_Average_Rating    1.007163
15              View_Count_previous_games    7.237026
16      video_potentiality_previous_games    7.238720
17                 

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


In [ ]:
data = segment_cocobi.copy()

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter
from patsy.contrasts import Treatment

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User acquisition'])

# 새로운 YouTube_Flag 생성: _youtube가 포함되지만 unsimilar_youtube 제외
youtube_columns = data.filter(like='_youtube').columns
youtube_columns = youtube_columns[~youtube_columns.str.contains('unsimilar')]
data['YouTube_Flag'] = data[youtube_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# 새로운 Games_Flag 생성: _games가 포함되지만 previous_games 제외
games_columns = data.filter(like='_games').columns
games_columns = games_columns[~games_columns.str.contains('previous')]
data['Games_Flag'] = data[games_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AD_LAN_original'] = data['AD_LAN']
data['segment_original'] = data['segment']

# 변수 이름의 공백을 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_')

# Check if 'segment' column exists before converting to dummy variables
if 'segment' in data.columns:
    data = pd.get_dummies(data, columns=['segment'], drop_first=False)
else:
    print("'segment' column not found, skipping dummy conversion for 'segment'.")

# Language_Game_Flag 적용 함수
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN_original'] == '한국어' or row['AD_LAN_original'] == '영어')
    if is_korean_or_english and row['video_potentiality_games'] > 0:
        return 1
    else:
        return 0

# Language_Game_Flag 적용
data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['Store_Visit','View_Count_games',
       'similarity_score', 'View_Count_youtube', 'Like_Count_youtube',
       'Comment_Count_youtube', 'video_potentiality_youtube',
       'video_potentiality_games', 'day+_youtube', 'day+_games',
       'Category_ID_youtube', 'Category_ID_games', 'similarity_score_games',
      'Total_Average_Rating','View_Count_previous_games', 'video_potentiality_previous_games',
       'day+_unsimilar_youtube', 'video_potentiality_unsimilar_youtube',
       'Comment_Count_Count_unsimilar_youtube', 'View_Count_unsimilar_youtube',
       'Like_Count_unsimilar_youtube','YouTube_Flag','Games_Flag','Language_Game_Flag']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars +
         list(data.filter(like='segment').columns)] if 'segment' in data.columns else data[independent_vars]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# 스케일링 적용
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
print("VIF가 낮은 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data['User_acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# 패널 데이터를 위한 필요한 독립 변수만 선택 (low_vif_features 및 원본 AppName, Country, AD_LAN 사용)
panel_data = data[['Date','segment_original', 'Permanent_Component', 'Temporary_Component'] + low_vif_features]

# 결측값 제거
panel_data = panel_data.dropna()

# 변수명에서 +를 _로 교체
panel_data.columns = panel_data.columns.str.replace('+', '_')
low_vif_features = [feature.replace('+', '_') for feature in low_vif_features]


# 고정효과 패널 회귀 분석 (Permanent Component 분석) - segment와 Language_Game_Flag 상호작용 추가
model_perm = smf.ols('Permanent_Component ~ ' + ' + '.join(low_vif_features) +
                     ' + C(segment_original, Treatment(reference=3)) * Language_Game_Flag', data=panel_data).fit()

# 고정효과 패널 회귀 분석 (Temporary Component 분석) - segment와 Language_Game_Flag 상호작용 추가
model_temp = smf.ols('Temporary_Component ~ ' + ' + '.join(low_vif_features) +
                     ' + C(segment_original, Treatment(reference=3)) * Language_Game_Flag', data=panel_data).fit()

# 결과 출력
print("고정효과 패널 분석 결과 (Permanent Component):")
print(model_perm.summary())

print("\n고정효과 패널 분석 결과 (Temporary Component):")
print(model_temp.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


VIF 결과:
                                   Feature         VIF
0                                   const         NaN
1                             Store_Visit    1.013292
2                        View_Count_games   12.527275
3                        similarity_score    6.891993
4                      View_Count_youtube    1.817350
5                      Like_Count_youtube    1.080320
6                   Comment_Count_youtube         NaN
7              video_potentiality_youtube    1.773818
8                video_potentiality_games   13.497815
9                            day+_youtube    4.235413
10                             day+_games   11.187053
11                    Category_ID_youtube    2.748766
12                      Category_ID_games   92.749093
13                 similarity_score_games   11.752945
14                   Total_Average_Rating    1.007163
15              View_Count_previous_games    7.237026
16      video_potentiality_previous_games    7.238720
17                 

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


In [ ]:
data = segment_cocobi.copy()

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter
from patsy.contrasts import Treatment

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User acquisition'])

# 새로운 YouTube_Flag 생성: _youtube가 포함되지만 unsimilar_youtube 제외
youtube_columns = data.filter(like='_youtube').columns
youtube_columns = youtube_columns[~youtube_columns.str.contains('unsimilar')]
data['YouTube_Flag'] = data[youtube_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# 새로운 Games_Flag 생성: _games가 포함되지만 previous_games 제외
games_columns = data.filter(like='_games').columns
games_columns = games_columns[~games_columns.str.contains('previous')]
data['Games_Flag'] = data[games_columns].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AD_LAN_original'] = data['AD_LAN']
data['segment_original'] = data['segment']

# 변수 이름의 공백을 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_')

# Check if 'segment' column exists before converting to dummy variables
if 'segment' in data.columns:
    data = pd.get_dummies(data, columns=['segment'], drop_first=False)
else:
    print("'segment' column not found, skipping dummy conversion for 'segment'.")

# Language_Game_Flag 적용 함수
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN_original'] == '한국어' or row['AD_LAN_original'] == '영어')
    if is_korean_or_english and row['video_potentiality_games'] > 0:
        return 1
    else:
        return 0

# Language_Game_Flag 적용
data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['Store_Visit','View_Count_games',
       'similarity_score', 'View_Count_youtube', 'Like_Count_youtube',
       'Comment_Count_youtube', 'video_potentiality_youtube',
       'video_potentiality_games', 'day+_youtube', 'day+_games',
       'Category_ID_youtube', 'Category_ID_games', 'similarity_score_games',
      'Total_Average_Rating','View_Count_previous_games', 'video_potentiality_previous_games',
       'day+_unsimilar_youtube', 'video_potentiality_unsimilar_youtube',
       'Comment_Count_Count_unsimilar_youtube', 'View_Count_unsimilar_youtube',
       'Like_Count_unsimilar_youtube','YouTube_Flag','Games_Flag','Language_Game_Flag']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars +
         list(data.filter(like='segment').columns)] if 'segment' in data.columns else data[independent_vars]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# 스케일링 적용
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
print("VIF가 낮은 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data['User_acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# 패널 데이터를 위한 필요한 독립 변수만 선택 (low_vif_features 및 원본 AppName, Country, AD_LAN 사용)
panel_data = data[['Date','segment_original', 'Permanent_Component', 'Temporary_Component'] + low_vif_features]

# 결측값 제거
panel_data = panel_data.dropna()

# 변수명에서 +를 _로 교체
panel_data.columns = panel_data.columns.str.replace('+', '_')
low_vif_features = [feature.replace('+', '_') for feature in low_vif_features]


# 고정효과 패널 회귀 분석 (Permanent Component 분석) - segment와 Language_Game_Flag 상호작용 추가
model_perm = smf.ols('Permanent_Component ~ ' + ' + '.join(low_vif_features) +
                     ' + C(segment_original, Treatment(reference=4)) * Language_Game_Flag', data=panel_data).fit()

# 고정효과 패널 회귀 분석 (Temporary Component 분석) - segment와 Language_Game_Flag 상호작용 추가
model_temp = smf.ols('Temporary_Component ~ ' + ' + '.join(low_vif_features) +
                     ' + C(segment_original, Treatment(reference=4)) * Language_Game_Flag', data=panel_data).fit()

# 결과 출력
print("고정효과 패널 분석 결과 (Permanent Component):")
print(model_perm.summary())

print("\n고정효과 패널 분석 결과 (Temporary Component):")
print(model_temp.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


VIF 결과:
                                   Feature         VIF
0                                   const         NaN
1                             Store_Visit    1.013292
2                        View_Count_games   12.527275
3                        similarity_score    6.891993
4                      View_Count_youtube    1.817350
5                      Like_Count_youtube    1.080320
6                   Comment_Count_youtube         NaN
7              video_potentiality_youtube    1.773818
8                video_potentiality_games   13.497815
9                            day+_youtube    4.235413
10                             day+_games   11.187053
11                    Category_ID_youtube    2.748766
12                      Category_ID_games   92.749093
13                 similarity_score_games   11.752945
14                   Total_Average_Rating    1.007163
15              View_Count_previous_games    7.237026
16      video_potentiality_previous_games    7.238720
17                 

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
